<a href="https://colab.research.google.com/github/RapidFireAI/rapidfireai/blob/main/tutorial_notebooks/COLAB_QUICKSTART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 RapidFire AI - Google Colab Quickstart

This notebook demonstrates how to run RapidFire AI in Google Colab.

## What You'll Learn
- ✅ Install RapidFire AI in Colab
- ✅ Start RapidFire services with Cloudflare Tunnel (free, public URLs)
- ✅ Access the RapidFire Dashboard, MLflow UI, and API
- ✅ Run a simple fine-tuning experiment

## Prerequisites
- Google Colab account (free tier works!)
- GPU runtime enabled (Runtime → Change runtime type → GPU)

---

## 1️⃣ Check GPU Availability

In [ ]:
!nvidia-smi

## 2️⃣ Install RapidFire AI

This will install RapidFire AI and all dependencies.

In [ ]:
# Install RapidFire AI from PyPI
!pip install -q rapidfireai

# Initialize RapidFire (installs additional dependencies)
!rapidfireai init

## 3️⃣ Start RapidFire Services

This cell will:
- Start MLflow server (port 5002)
- Start Dispatcher API (port 8080)
- Start Frontend Dashboard (port 3000)
- Expose all services using **Cloudflare Tunnel** (free, public URLs)

**Note**: 
- This cell will keep running
- Look for `.trycloudflare.com` URLs in the output
- URLs are publicly accessible (great for sharing with team)
- Takes ~30 seconds to start and expose all services

In [ ]:
# Start RapidFire in Colab mode
# Uses Cloudflare Tunnel by default (free, public URLs)
!rapidfireai start --colab

### Alternative: Using Native Colab Port Forwarding

If you prefer Colab's native port forwarding (authenticated, private URLs), use the Python API.

**Important**: Native forwarding only works when called from a notebook cell, not from CLI.

Run these cells instead of the one above:

In [ ]:
# Step 1: Start services in background (without tunneling)
import subprocess
import os
import time

# Set hosts to 0.0.0.0 for external access
os.environ['RF_MLFLOW_HOST'] = '0.0.0.0'
os.environ['RF_API_HOST'] = '0.0.0.0'
os.environ['RF_FRONTEND_HOST'] = '0.0.0.0'

# Start services in background
print("Starting RapidFire services in background...")
proc = subprocess.Popen(
    ['rapidfireai', 'start'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print("Waiting for services to start (30 seconds)...")
time.sleep(30)
print("✅ Services should be running now")

In [ ]:
# Step 2: Expose ports using native Colab forwarding
# This MUST run from a notebook cell (won't work from CLI)
from rapidfireai.utils.colab_helper import expose_rapidfire_services

print("Exposing services via native Colab port forwarding...\n")
urls = expose_rapidfire_services(
    method='native',
    mlflow_port=5002,
    dispatcher_port=8080,
    frontend_port=3000
)

print("\n📋 Services exposed:")
for service, url in urls.items():
    if url:
        print(f"  {service.upper()}: {url}")

### 🔄 Cloudflare vs Native: Which to Use?

| Feature | Cloudflare (Default) | Native Colab |
|---------|---------------------|-------------|
| **Setup** | One command | Two steps |
| **URLs** | Public (shareable) | Private (authenticated) |
| **Speed** | ~30 sec startup | Instant |
| **Access** | Anyone with URL | Only you |
| **Use Case** | Team collaboration | Personal use |

**Recommendation**: Use Cloudflare (default) unless you need private URLs.

## 4️⃣ Access Your Services

After running the cell above, you should see three URLs:

1. **RapidFire Dashboard** - Main UI for:
   - Viewing experiment runs
   - Real-time metrics
   - Interactive Control Operations (stop/resume/clone)

2. **MLflow Tracking UI** - For:
   - Detailed experiment tracking
   - Parameter comparison
   - Model artifacts

3. **Dispatcher API** - REST API endpoint

Click the URLs to open them in new tabs!

## 5️⃣ Run a Simple Experiment

Now let's create a minimal fine-tuning experiment to test everything works.

**Note**: Open a new notebook or create new cells for this, as the services need to keep running.

In [ ]:
# Minimal example - see other tutorial notebooks for full examples
from rapidfireai import Experiment
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

# Create experiment
exp = Experiment("colab_quickstart_test")

# Simple configuration
config = {
    'trainer_type': 'SFT',
    'training_args': {
        'learning_rate': 1e-5,
        'per_device_train_batch_size': 2,
        'max_steps': 10,  # Short for demo
    }
}

# Load tiny dataset
dataset = load_dataset('imdb', split='train[:50]')

# Model creation function
def create_model(config):
    model = AutoModelForCausalLM.from_pretrained('gpt2')
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Run experiment
print("🚀 Starting experiment...")
exp.run_fit(
    param_config=config,
    create_model_fn=create_model,
    train_dataset=dataset,
    eval_dataset=dataset.select(range(10)),
    num_chunks=2,
    seed=42
)

print("\n✅ Experiment complete! Check the dashboard for results.")

## 🎯 Next Steps

Now that you have RapidFire running in Colab, try:

### 1. Explore Other Tutorials
- `rf-tutorial-sft-chatqa-lite.ipynb` - Supervised fine-tuning
- `rf-tutorial-dpo-alignment-lite.ipynb` - DPO training
- `rf-tutorial-grpo-mathreasoning-lite.ipynb` - GRPO for math

### 2. Try AutoML
```python
from rapidfireai.automl import GridSearch, RFModelConfig

config = RFModelConfig(
    training_args={
        'learning_rate': [1e-4, 1e-5, 1e-6],
        'batch_size': [8, 16]
    }
)

grid = GridSearch(configs=[config])
exp.run_fit(param_config=grid, ...)
```

### 3. Use Interactive Control
In the dashboard:
- Stop/resume runs mid-training
- Clone runs with modified hyperparameters
- Warm-start from previous checkpoints
- Compare results in real-time

---

## 📚 Resources

- **Documentation**: https://rapidfire-ai-oss-docs.readthedocs-hosted.com/
- **GitHub**: https://github.com/RapidFireAI/rapidfireai
- **Full Colab Guide**: [COLAB_SETUP.md](https://github.com/RapidFireAI/rapidfireai/blob/main/tutorial_notebooks/COLAB_SETUP.md)

---

## ⚠️ Troubleshooting

**Services not starting?**
```bash
# Check if ports are in use
!lsof -ti :5002 :8080 :3000
```

**Cloudflare tunnel failing?**
```python
# Try ngrok instead (requires auth token)
import os
os.environ['RF_NGROK_TOKEN'] = 'your_token'
!rapidfireai start --colab --tunnel ngrok
```

**Out of memory?**
- Use smaller models
- Reduce batch size
- Enable gradient checkpointing

---

**Happy fine-tuning! 🎉**